In [4]:
import requests as req
from bs4 import BeautifulSoup as bsp
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from seleniumwire import webdriver  # blinker == 1.7.0
import requests
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from fake_useragent import UserAgent
from googlesearch import search

base = 'https://www.amazon.com'
thirdWheel = '/s?k='

In [ ]:
# session = requests.Session()
# session.headers.update({"User-Agent": UserAgent().random})
driver = webdriver.Chrome()
# driver = webdriver.Firefox(service = service, options = options)
driver.get(base)
#   It will scroll to right above the footer of the page then scoll to the top then back to the bottom untill there is no new items being loaded
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(10) 
html = driver.page_source
driver.quit()          
soup = bsp(html, 'lxml')
boxes = soup.find_all('a', class_ = 'a-link-normal _fluid-quad-image-label-v2_style_centerImage__30wh- aok-block image-window')
print(boxes)
links = []
urls = []
categ = []
for box in boxes:
    actual = box.get('href')
    url = base + actual
    print("Url : ", url)
    urls.append(url)
    som = box.get('aria-label')
    link = som.replace(' ', '+')
    link = base + thirdWheel + link
    print("Links : ", link)
    links.append(link)
    categ.append(som)
    print(som)


In [6]:
def get_valid_proxies():
    proxy_list_url = 'https://free-proxy-list.net/'
    response = requests.get(proxy_list_url)
    soup = bsp(response.text, 'html.parser')
    proxy_data = []
    rows = soup.find_all('tr')[1:]
    for row in rows:
        columns = row.find_all('td')
        if len(columns) >= 8:
            ip_address = columns[0].text.strip()
            google_enabled = columns[5].text.strip().lower() == 'yes'
            https_enabled = columns[6].text.strip().lower() == 'yes'
            last_checked = columns[7].text.strip()
            if (last_checked.endswith('mins ago') and int(last_checked.split(' ')[0]) < 15) or last_checked.endswith('hours ago'):
                if google_enabled or https_enabled:
                    proxy_data.append({'ip_address': ip_address, 'google_enabled': google_enabled, 'https_enabled': https_enabled})

    return proxy_data


In [7]:
def rotate_user_agent(proxy):
    if proxy:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'http': f'http://{proxy}',
            'https': f'https://{proxy}'
        }
    else:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    return headers

In [ ]:
products = []
categories = []
cat = 0
me = False
# proxies = get_valid_proxies()
for url in urls:
    total_samples = 0
    myurl = url
    while myurl:
        # proxy = proxies[total_samples % len(proxies)] if proxies else None
        options = {
            'headers' : {
                "User-Agent": UserAgent().random
            }
        }
        print("VISITING : ", myurl)
        # sdsd = Options()
        # sdsd.add_argument("--headless")  # Run in background
        # sdsd.add_argument("--disable-gpu")
        # sdsd.add_argument("--no-sandbox")
        # sdsd.add_argument("--disable-dev-shm-usage")
        driver = webdriver.Chrome(seleniumwire_options = options)
        # driver = webdriver.Firefox()
        driver.get(myurl)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(10)
        html = driver.page_source
        driver.quit()
        # headers = rotate_user_agent(proxy)
        # response = req.get(myurl, headers = headers)
        # session = requests.Session()
        # session.headers.update({"User-Agent": UserAgent().random})
        # response = req.get(myurl, headers = {"User-Agent": UserAgent().random})
        # if response.status_code == 200:
            # soup = bsp(response.text, 'lxml')
        soup = bsp(html, 'lxml')
        # print(soup)
        productes = soup.find_all('a', class_ = 'a-link-normal s-line-clamp-4 s-link-style a-text-normal')
        
        if not productes:
            productes = soup.find_all('a' , class_= 'a-link-normal s-line-clamp-2 s-link-style a-text-normal')
        print(productes)
        for product in productes:
            product = base + product.get('href')
            products.append(product)
            if cat in range(len(categ)):
                categories.append(categ[cat])
            print(product)
            csv_file = 'Amazon.csv'
            info = {"Products": products, "Categories": categories}
            new_df = pd.DataFrame(info)
            if not me:
                new_df.to_csv(csv_file, mode='a', index = False)
                me = True
            else:
                new_df.to_csv(csv_file, mode='a', header=False, index = False)
            print("|||||||||||||||||||||||||||| Data appended successfully with continued index. |||||||||||||||||||||||||||")
        
        total_samples += 1
        nextP = soup.find('a', class_ = 's-pagination-item s-pagination-next s-pagination-button s-pagination-button-accessibility s-pagination-separator')
        if nextP:
            myurl = base + nextP.get('href')
        if total_samples >= 3:
            break
    cat += 1



In [ ]:
agent = {
    "Main_Category": [],
    "Title": [],
    "AvgRating": [],
    "RatingNo": [],
    "Features": [[]],
    # "Description": [[]],
    "Price": [],
    "Images": [[]],
    "Videos": [[]],
    "Store": [],
    # "Categories": [],
    "Details": [[]],
    # "ParentAsin": [],
    # "BoughtTogether": [[]]
}

comment = {
    
}

csv_file = 'G:\\Amazon.csv'
df = pd.readcsv(csv_file)
samples = 0
for row in df.iterrows():
    options = {
            'headers' : {
                "User-Agent": UserAgent().random
            }
        }
    link = row['Products']
    print("VISITING : ", link)
    driver = webdriver.Chrome(seleniumwire_options = options)
    driver.get(link)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(10)
    html = driver.page_source
    driver.quit()
    soup = bsp(html, 'lxml')

    feat = []
    img = []
    vid = []


    # extracting Category
    category = row['Categories']

    # extracting Title
    titleBoard = soup.find('span', id = 'productTitle')
    title = titleBoard.text

    # extracting ratings
    ratingBoard = soup.find('div', class_ = 'averageCustomerReviews')
    # extracting average rating
    avgRatingBoard = ratingBoard.find('span', class_ = 'a-size-base a-color-base')
    avgRating = avgRatingBoard.text.strip()
    # extracting rating number
    ratingNoBoard = ratingBoard.find('span', id = 'acrCustomerReviewText')
    ratingNo = ratingNoBoard.text.strip()

    # extracting price
    priceBoard = soup.find('div', class_ = 'a-section a-spacing-none aok-align-center aok-relative').find('span', class_ = 'aok-offscreen')
    price = priceBoard.text.strip()

    # exracting media
    mediaBoard = soup.find('ul', class_ = 'a-unordered-list a-nostyle a-button-list a-vertical a-spacing-top-micro regularAltImageViewLayout')
    media = mediaBoard.find_all('li')

    # extracting the store
    storeBoard = soup.find('tr', class_ = 'a-spacing-small po-brand').find('td', class_ = 'a-span9')
    store = storeBoard.text.strip()

    # extracting features
    featuresBoard = soup.find('ul', class_ = 'a-unordered-list a-vertical a-spacing-mini').find_all('il')
    features = featuresBoard

    # extracting details
    detailsBoard = soup.find('div', class_ = 'a-expander-content a-expander-section-content a-section-expander-inner').find('table', class_ = 'a-keyvalue prodDetTable').find_all('tr')
    details = detailsBoard

    detailsBoardB = soup.find('div', id = 'detailBullets_feature_div').find_all('li')
    detailsB = detailsBoardB

    detailsBoardC = soup.find('table', id = 'productDetails_detailBullets_sections1').find_all('tr')

    asinBorad = soup.find('tr', text=lambda x: x.strip() == 'ASIN', class_='prodDetSectionEntry')
    asin = asinBoard.text.strip();

    


In [ ]:

from datetime import datetime

# Define the date strings
date_str1 = "June 22, 2016"
date_str2 = "September 30, 2023"

# Convert the date strings to datetime objects
date1 = datetime.strptime(date_str1, "%B %d, %Y")
date2 = datetime.strptime(date_str2, "%B %d, %Y")

# Compare the dates
if date1 > date2:
    print(f"{date_str1} is greater than {date_str2}")
else:
    print(f"{date_str1} is not greater than {date_str2}")